**CCT - Final Project**

In [ ]:
# Importing the libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, LabelEncoder
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor
from __future__  import print_function #adds compatibility with new versions of Python
%matplotlib inline 
#it needs to have two underscore for 'future', if not it thorws an error: 
#ModuleNotFoundError: No module named '_future_'

import warnings
warnings.filterwarnings("ignore") #This command ignores the warning 

#other llbraries and dependencies were added along the notebook. 

In [ ]:
#Dependecies used in this notebook.
#To install, please remove the hashtag.

#pip install panda
#pip install seaborn
#pip install plotly
#pip install matplotlib
#pip install scikit-learn
#pip install spacy
#pip install wordcloud
#pip install stopwords
#pip install tensorflow
#pip install googletrans==4.0.0rc1

Index of Datasets:
<p> 1) Listings </p>
<p> 2) Review </p>
<p> 3) Calendar </p>
<p> 4) Airbnb_listing </p>
<p> 5) Airbnb_listing1 </p>
<p> 6) Rate </p>

<p>Some of the datasets are for trends and whether or not features influence reviews that influence occupancy that generates higher ROI. 
The datasets for ML Listing ad Rate, they are sample data of Dublin to help identify the price and occupancy rate in the future.</p>

In [ ]:
#Importing the dataset #1

#The dataset chosen for this project is listings for Airbnb 

listing = pd.read_csv('listings.csv')


In [ ]:
#Importing the dataset #2

#The dataset chosen for this project is reviews for Airbnb 

review = pd.read_csv('reviews.csv')


In [ ]:
#Importing the dataset # 3

#The dataset chosen for this project is calendar for Airbnb

calendar = pd.read_csv('calendar.csv')


In [ ]:
#Importing the dataset # 4

#The dataset chosen for this project is the Dublin listings from OpenData 

airbnb_listing_1 = pd.read_csv('airbnb-listings.csv', sep=";")


In [ ]:
#Importing the dataset # 5

#The dataset chosen for this project is Dublin listings from OpenData 

airbnb_listing_2 = pd.read_csv('airbnb-opendata.csv', sep=";")

In [ ]:
#Importing the dataset # 6

#The dataset chosen for this project is Dublin rate from Airbtics 
rate = pd.read_csv('rate.csv')

#this dataset is a sample, however it contains a lot of information. 
#It needed to be fixed as it didn't have the columns set right.


**Dataset 1: Listings**


Data Understanding & Cleaning

In [ ]:
listing.info()

In [ ]:
# Drop the data that are not of interest and/or causing privacy issues
listing.drop(['id','host_name','last_review'], axis=1, inplace=True)
# Visualize the first 5 rows
listing.head(100)

In [ ]:
listing.isnull().sum()

In [ ]:
listing.info()

In [ ]:
listing.drop(['neighbourhood_group', 'license'], axis=1, inplace=True)
#listing.drop(['neighbourhood_group', 'license', 'name'], axis=1, inplace=True)

**Data Visualisation - EDA Listing**

In [ ]:
# Visualizing the distribution for every "feature"
listing.hist(edgecolor="black", linewidth=1.2, figsize=(30, 30));

In [ ]:
plt.figure(figsize=(30, 30))
sns.pairplot(listing, height=5, diag_kind="hist")

#latitude and longitude have a normal distribution, most of the hosts are concetrated in specific area.
#reviews_per_month has a lot of outliers, because of the missing values could be filled by mean() and mode()
#most the hosts have a price under 1000

In [ ]:
# Another way to visualize the data is to use FacetGrid to plot multiple kedplots on one plot

fig = sns.FacetGrid(listing, hue="neighbourhood", aspect=4, height=10)
fig.map(sns.kdeplot, 'host_id', shade=True)
oldest = listing['host_id'].max()
fig.set(xlim=(0, oldest))
sns.set(font_scale=5)
fig.add_legend()

In [ ]:
sns.set(font_scale=1.5)
sns.catplot("room_type", data=listing, kind="count", height=8)

In [ ]:
# Another way to visualize the data is to use FacetGrid to plot multiple kde plots on one plot

fig = sns.FacetGrid(listing, hue="room_type", aspect=4, height=10)
fig.map(sns.kdeplot, 'host_id', shade=True)
oldest = listing['host_id'].max()
fig.set(xlim=(0, oldest))
sns.set(font_scale=5)
fig.add_legend()

In [ ]:
sns.set(font_scale=1.5)
plt.figure(figsize=(12, 8))
listing.host_id.hist(bins=100)

In [ ]:
data = listing.neighbourhood.value_counts()[:10]
plt.figure(figsize=(10, 6))
x = list(data.index)
y = list(data.values)
x.reverse()
y.reverse()

plt.title("Most Popular Neighbourhood")

plt.barh(x, y)

In [ ]:
listing['neighbourhood'].value_counts()

#Dublin City has 5334 properties listed out of 6977 in total
#Dun Laoghaire has 742
#Fingal has 616 and South Dublin represents the lower portion with only 285 properties listed.

In [ ]:
#Check property availability per neighbourhood
plt.figure(figsize=(14,6));
listing.groupby(['neighbourhood'])['availability_365'].mean().sort_values(ascending=False).plot(kind='bar');
plt.title('Availability Percentage by Neighbourhood');
plt.ylabel('Availability in 365 days');

In [ ]:
listing['number_of_reviews'].value_counts()

#1450 out of 6977 Properties have 0 reviews. 
#653 properties have at least 1 review.
#463 properties have more than 1 review.

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(listing.longitude, listing.latitude, c=listing.availability_365, cmap='summer', edgecolor='black', linewidth=1, alpha=0.75)

cbar = plt.colorbar()
cbar.set_label('availability_365')


plt.figure(figsize=(12, 8))
plt.scatter(listing.longitude, listing.latitude, c=listing.price, cmap='summer', edgecolor='black', linewidth=1, alpha=0.75)

cbar = plt.colorbar()
cbar.set_label('Price $')

In [ ]:
print(f"Average of price per night : ${listing.price.mean():.2f}")
print(f"Maximum price per night : ${listing.price.max()}")
print(f"Minimum price per night : ${listing.price.min()}")

In [ ]:
# group the listings by neighbourood and get the average price
listing.groupby('neighbourhood')['price'].mean().sort_values(ascending=False)

In [ ]:
(listing.groupby('neighbourhood')['price'].mean().sort_values(ascending=False)).plot(kind="bar", figsize=(16,8));
plt.title("Average Price per Neighbourhood");
plt.ylabel('Average Price');

#Entire homes or apartments seem to be the most expensive type of properties in Ireland,
#followed by Hotel rooms, private rooms and shared rooms (hostels).

In [ ]:
#really expensive house

listing[listing.price == 1173721]

In [ ]:
#really cheap houses

listing[listing.price <10]

In [ ]:
#Limit price to 2000
ax = sns.boxplot( y="price", data=listing)
ax.set_ylim([0, 2000])

In [ ]:
# Computing IQR
q1 = listing['price'].quantile(0.25)
q3 = listing['price'].quantile(0.75)
iqr = q3 - q1
print(iqr)

print(q3+(1.5*iqr))

outliers= listing.loc[listing['price'] > q3+(1.5*iqr)]

In [ ]:
listing_without_outliers = listing.loc[listing['price'] <= q3+(1.5*iqr)]

#Boxplot
ax = sns.boxplot( y="price", data=listing_without_outliers)

In [ ]:
sub_1=listing_without_outliers[listing_without_outliers.price < 500]

import matplotlib.image as mpimg
%matplotlib inline

vis_1=sub_1.plot(kind='scatter', x='longitude', y='latitude', label='neighbourhood', c='price',
                  cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4, figsize=(10,8))
vis_1.legend()

In [ ]:
# correlation matrix
sns.set(font_scale=3)
plt.figure(figsize=(30, 20))
sns.heatmap(listing.corr(), annot=True)

In [ ]:
plt.figure(figsize=(30, 30))
sns.set(font_scale=1.5)
i = 1
for column in listing.columns:
    if listing[column].dtype == "float64" or listing[column].dtype == "int64":
        plt.subplot(5, 5, i)
        listing.corr()[column].sort_values().plot(kind="barh")
        i += 1

In [ ]:
top_host= listing.host_id.value_counts().head(10)
top_host

top_host=pd.DataFrame(top_host)
top_host.reset_index(inplace=True)
top_host.rename(columns={'index':'Host_ID', 'host_id':'P_Count'}, inplace=True)
top_host

In [ ]:
top_host_check= listing.calculated_host_listings_count.max()
top_host_check

In [ ]:
barchart_host=sns.barplot(x="Host_ID", y="P_Count", data=top_host,
                 palette='Blues_d')
barchart_host.set_title('Hosts with the most listings in Dublin')
barchart_host.set_ylabel('Count of listings')
barchart_host.set_xlabel('Host IDs')
barchart_host.set_xticklabels(barchart_host.get_xticklabels(), rotation=45)

In [ ]:
listing['availability_365'].value_counts()

#4237 properties have 0 days available.
#Only 2740 properties out of 6977 had more than 1 day available for rental at the time this data was collected. 
#186 properties are available for at least 358 days a year.

In [ ]:
scatter_avail=listing.plot(kind='scatter', x='longitude', y='latitude', label='availability_365', c='price',
                  cmap=plt.get_cmap('jet'), colorbar=True, alpha=0.4, figsize=(10,8))
scatter_avail.legend()

In [ ]:
listing.fillna({'reviews_per_month':0}, inplace=True)

In [ ]:
(listing[['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']]
 .describe())

In [ ]:
listing = listing.loc[listing['price'] > 0]

In [ ]:
listing.describe()

In [ ]:
listing.head()

In [ ]:
# Recode data as categorical
listing_encoded = listing.copy()
listing_encoded['minimum_nights'] = pd.qcut(listing['minimum_nights'], q=2, labels=["minimum_nights_low", "minimum_nights_high"])
listing_encoded['number_of_reviews'] = pd.qcut(listing['number_of_reviews'], q=3, labels=["number_of_reviews_low", "minimum_nights_medium", "number_of_reviews_high"])
listing_encoded['reviews_per_month'] = pd.qcut(listing['reviews_per_month'], q=2, labels=["reviews_per_month_low", "reviews_per_month_high"])
listing_encoded['calculated_host_listings_count'] = pd.cut(listing['calculated_host_listings_count'], 
                                                bins=[0, 2, 327],
                                                labels=["calculated_host_listings_count_low", "calculated_host_listings_count_high"])

#NEEDS TO BE FIXED
#listing_encoded['availability_365'] = pd.qcut(listing['availability_365'], q=2, labels=["availability_low", "availability_high"])


In [ ]:
listing_encoded.isnull().sum()

In [ ]:
sns.set_palette("muted")
from pylab import *
f, ax = plt.subplots(figsize=(8, 6))

subplot(2,3,1)
sns.distplot(listing['price'])

subplot(2,3,2)
sns.distplot(listing['minimum_nights'])

subplot(2,3,3)
sns.distplot(listing['number_of_reviews'])

subplot(2,3,4)
sns.distplot(listing['reviews_per_month'])

subplot(2,3,5)
sns.distplot(listing['calculated_host_listings_count'])

subplot(2,3,6)
sns.distplot(listing['availability_365'])

plt.tight_layout() # avoid overlap of plotsplt.draw()

In [ ]:
title = 'Properties per Neighbourhood'
chart = sns.countplot(listing['neighbourhood'])

chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

plt.title(title)
plt.ioff()

In [ ]:
title = 'Properties per Room Type'
chart_prop = sns.countplot(listing['room_type'])
chart_prop.set_xticklabels(chart.get_xticklabels(), rotation=90) #add 90 degree to the neighbourhood.
plt.title(title)
plt.ioff()

In [ ]:
plt.figure(figsize=(20,10))
title = 'Correlation matrix of numerical variables'
sns.heatmap(listing.corr(), square=True, cmap='RdYlGn')
plt.title(title)
plt.ioff()

In [ ]:
corr = listing.corr()

In [ ]:
corr.style.background_gradient(cmap = 'coolwarm')

#Strong Correlations were found between:
#The number_of_reviews_ltm and reviews_per_month columns (0.75)
#The number_of_reviews and reviews_per_month (0.55)
#No further important correlations were found.

In [ ]:
title = 'Neighbourhood Location'
plt.figure(figsize=(10,6))
sns.scatterplot(listing.longitude,listing.latitude,hue=listing.neighbourhood).set_title(title)
plt.ioff()

title = 'Room type location per Neighbourhood'
plt.figure(figsize=(10,6))
sns.scatterplot(listing.longitude,listing.latitude,hue=listing.room_type).set_title(title)
plt.ioff()

In [ ]:
listing.info()


In [ ]:
#initializing empty list where we are going to put our name strings
_names_=[]
#getting name strings from the column and appending it to the list
for name in listing.name:
    _names_.append(name)
#setting a function that will split those name strings into separate words   
def split_name(name):
    spl=str(name).split()
    return spl
#initializing empty list where we are going to have words counted
_names_for_count_=[]
#getting name string from our list and using split function, later appending to list above
for x in _names_:
    for word in split_name(x):
        word=word.lower()
        _names_for_count_.append(word)

In [ ]:
#we are going to use counter
from collections import Counter
#let's see top 25 used words by host to name their listing
_top_25_w=Counter(_names_for_count_).most_common()
_top_25_w=_top_25_w[0:25]

In [ ]:
#now let's put our findings in dataframe for further visualizations
sub_w=pd.DataFrame(_top_25_w)
sub_w.rename(columns={0:'Words', 1:'Count'}, inplace=True)

In [ ]:
viz_5=sns.barplot(x='Words', y='Count', data=sub_w)
viz_5.set_title('Counts of the top 25 used words for listing names')
viz_5.set_ylabel('Count of words')
viz_5.set_xlabel('Words')
viz_5.set_xticklabels(viz_5.get_xticklabels(), rotation=90)

In [ ]:
sub_1=listing.loc[listing['neighbourhood'] == 'Dn Laoghaire-Tathdown']
price_sub1=sub_1[['price']]
#Manhattan
sub_2=listing.loc[listing['neighbourhood'] == 'South Dublin']
price_sub2=sub_2[['price']]
#Queens
sub_3=listing.loc[listing['neighbourhood'] == 'Dublin City']
price_sub3=sub_3[['price']]
#Staten Island
sub_4=listing.loc[listing['neighbourhood'] == 'Fingal']
price_sub4=sub_4[['price']]

#putting all the prices' dfs in the list
price_list_by_n=[price_sub1, price_sub2, price_sub3, price_sub4]

In [ ]:
p_l_b_n_2=[]
#creating list with known values in neighbourhood_group column
nei_list=['Dn Laoghaire-Tathdown', 'South Dublin', 'Dublin City', 'Fingal']
#creating a for loop to get statistics for price ranges and append it to our empty list
for x in price_list_by_n:
    i=x.describe(percentiles=[.25, .50, .75])
    i=i.iloc[3:]
    i.reset_index(inplace=True)
    i.rename(columns={'index':'Stats'}, inplace=True)
    p_l_b_n_2.append(i)
#changing names of the price column to the area name for easier reading of the table    
p_l_b_n_2[0].rename(columns={'price':nei_list[0]}, inplace=True)
p_l_b_n_2[1].rename(columns={'price':nei_list[1]}, inplace=True)
p_l_b_n_2[2].rename(columns={'price':nei_list[2]}, inplace=True)
p_l_b_n_2[3].rename(columns={'price':nei_list[3]}, inplace=True)

#finilizing our dataframe for final view    
stat_df=p_l_b_n_2
stat_df=[df.set_index('Stats') for df in stat_df]
stat_df=stat_df[0].join(stat_df[1:])
stat_df

In [ ]:
#creating a sub-dataframe with no extreme values / less than 500
sub_6=listing[listing.price < 500]
#using violinplot to showcase density and distribtuion of prices 
viz_2=sns.violinplot(data=sub_6, x='neighbourhood', y='price')
viz_2.set_title('Density and distribution of prices for each neighberhood')

In [ ]:
#View the top 10 properties most reviewed and get the average price per night
top_reviewed_listings=listing.nlargest(10,'number_of_reviews')
top_reviewed_listings

In [ ]:
price_avrg=top_reviewed_listings.price.mean()
print('Average price per night: {}'.format(price_avrg))


#These properties were the most reviewed from the dataset. 
#However, it is relevant to point out that they don't necessarily have the highest reviews. 
#In addition, another plot could be added in the future to check if there is a correlation between the highest reviews and the price per night. 
#90% of these properties are located in Dublin City. Which is another indication that Dublin City is indeed a relevant neighbourhood to consider when investing on a property or even when looking for accomodation.
#The average price per night of these properties is 77.70 euros

In [ ]:
listing.info()

**Models and predictions**

Price is the desired prediction.

In [ ]:
#Linear regression 
X = listing['price'].values.reshape(-1,1)
y = listing['availability_365'].values.reshape(-1,1)

In [ ]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

regressor = LinearRegression()  
regressor.fit(X_train, y_train) #train the model

In [ ]:
#Intercept
print(regressor.intercept_)
#Slope
print(regressor.coef_)

In [ ]:
#Predict
y_pred = regressor.predict(X_test)

In [ ]:
#actual value and predicted value
dfLinReg = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
dfLinReg

In [ ]:
import sklearn.metrics as metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


#The above linear regression model is not ideal.
#Changes to test size and random state did not impact the results

In [ ]:
#Multiple Regression

#Convert to Numeric
# creating instance of labelencoder
labelencoder = LabelEncoder()# Assigning numerical values and storing in another column
listing['room_type_Cat'] = labelencoder.fit_transform(listing['room_type'])
listing['city_Cat'] = labelencoder.fit_transform(listing['neighbourhood'])
listing.head()

In [ ]:
#multiple regression

X_ = listing[['calculated_host_listings_count', 'room_type_Cat', 
          'room_type_Cat', 'city_Cat']] # multiple variable regression. 
Y = listing['price']
 
# with sklearn
regr = LinearRegression()
regr.fit(X_, Y)

print('Intercept: ', regr.intercept_)
print('Coefficients: ', regr.coef_)

print (regr)

In [ ]:
y_pred2 = regr.predict(X_)

In [ ]:
dfmult= pd.DataFrame({'Actual': Y, 'Predicted': y_pred2.flatten()})
dfmult

In [ ]:
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y, y_pred2)))
print('Mean Absolute Error:', metrics.mean_absolute_error(Y, y_pred2))  
print('Mean Squared Error:', metrics.mean_squared_error(Y, y_pred2))  


In [ ]:
#comparison 
first20preds2=dfmult.head(20)
first20preds2.plot(kind='bar',figsize=(9,5))
plt.grid(which='major', linestyle='-', linewidth='0.3', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
#KNN
# create a list of conditions
conditions = [
    (listing['price'] < 100),
    (listing['price'] >= 100) & (listing['price'] < 250),
     (listing['price'] >= 250) & (listing['price'] < 600),
    (listing['price'] >= 600) ]

# create a list of the values we want to assign for each condition
values = ['economic', 'low-mid', 'high-mid','high']

# create a new column and use np.select to assign values to it using our lists as arguments
listing['price_range'] = np.select(conditions, values)

In [ ]:
listing['price_range'].value_counts()

In [ ]:
labelencoder = LabelEncoder()# Assigning numerical values and storing in another column
listing['price_rng_Cat'] = labelencoder.fit_transform(listing['price_range'])

In [ ]:
knn=listing[['minimum_nights','availability_365', 
         'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count',
         'room_type_Cat','city_Cat', 'price', 'price_rng_Cat']]

In [ ]:
knn.corr().style.background_gradient(cmap='magma')

In [ ]:
#conda install -c conda-forge scikit-learn


In [ ]:
#K-nearest neighbors (KNN)
##Best performing model

from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)

X1=knn.iloc[:, :-1].values
Y1=knn['price_rng_Cat'].values

# Split into training and test  
X_train, X_test, y_train, y_test = train_test_split( 
             X1, Y1, test_size = 0.4, random_state=1) 

#standardize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#fit the model
neigh.fit(X_train, y_train)

In [ ]:
# Predicted class
y_pred3=neigh.predict(X_test)

In [ ]:
KNNmod = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred3.flatten()})
KNNmod

In [ ]:
# Calculate the accuracy of the model 
print(neigh.score(X_test, y_test)) 

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred3)))

In [ ]:
#comparison 
first20preds3=KNNmod.head(20)
c2='darkkhaki', 'dimgray'
first20preds3.plot(kind='barh',figsize=(9,6), color=c2)
plt.grid(which='major', linestyle='-', linewidth='0.3', color='orange')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

**Dataset 2: Review**

This dataset was chosen specifically for the sentiment analysis. 

Data Understanding and Cleaning

In [ ]:
review.head()

In [ ]:
#Checking and removing if there are null values in the data.
review.isnull().sum()

In [ ]:
#Filling up null values with No review for generate word cloud.
review.fillna({'comments': 'Not available'}, inplace = True)

In [ ]:
review.isnull().sum()

In [ ]:
#Dropping columns that are not needed for the future model
review.drop(['id', 'listing_id','date','reviewer_id','reviewer_name'], axis=1, inplace=True)

In [ ]:
review.info()

**Sentiment Analysis**

Sentiment analysis is important to understand how much of impact the reviews are for a businness but more for this dataset it is important to discover positive or negative reviews of locations in Dublin.

It will use sentiment analysis from 1 to 5

In [ ]:
#!pip install torch torchvision torchaudio

In [ ]:
#conda install pytorch torchvision torchaudio -c pytorch

In [ ]:
#!pip install transformers requests beautifulsoup4

In [ ]:
#pip install transformers

In [ ]:
#pip install torchvision 

In [ ]:
#conda install statsmodels

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup

from statsmodels.stats.power import TTestIndPower

In [ ]:
#Instantiating the model

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
# Encoding and calculating sentiment

#A test of a negative review
tokens = tokenizer.encode('I did not like my stay, could be better', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
#Below is the result logits inside torch plus one to display the sentiment
int(torch.argmax(result.logits))+1

### Sample size: 
As this dataframe contains over 221.000 rows to create an sentiment for the future will take a while,so using a power analysis to discover the sample size will allow us to the best for our future model.

In [ ]:
#Start of power analysis
power_analysis = TTestIndPower()

In [ ]:
#Calculating the sample size
sample_size = power_analysis.solve_power(effect_size = 0.2, alpha = 0.05, power = 0.8, alternative = 'two-sided')

In [ ]:
#Results containing the sample
print('The sample size needed is: ', round(sample_size))

In [ ]:
#Selecting the first 500 rows of the dataframe to keep

review2 = review.head(500)

In [ ]:
#conda install -c conda-forge spacy

In [ ]:
#pip install -U pip setuptools wheel

In [ ]:
#pip install -U spacy

In [ ]:
#pip instal -U spacy

In [ ]:
#conda install tensorflow 

In [ ]:
#Scoring sentiment from dataframe
import re
import spacy
import nltk
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
#For creating the scoring the comments requires cleaning.
review2

In [ ]:
review2['comments'].iloc[0] 

In [ ]:
#Creating a method for scoring the sentiment

def sentiment_score(comments):
    tokens = tokenizer.encode(comments, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
#Diplay score sentiment of first row

sentiment_score(review2['comments'].iloc[1] )

In [ ]:
#Applying the scores to the dataframe.

review2['sentiment'] = review2['comments'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
review2

In [ ]:
#Tokenization for prediction with sentiment
#Cleaning method for tokenization

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

review2['Comments_processed'] = review2.comments.apply(lambda x: clean_text(x))

In [ ]:
#Show comments processed

review2.head()

In [ ]:
#Find the mean of all the comments
review2.Comments_processed.apply(lambda x: len(x.split(" "))).mean()

In [ ]:
#Creating model for prediction


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 6000
tokenizer = Tokenizer()
tokenizer.fit_on_texts(review2['Comments_processed'])
list_tokenized_train = tokenizer.texts_to_sequences(review2['Comments_processed'])

maxlen = 130
X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = review2['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dense(20, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_train,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
#Word Cloud
#Visualization of most use words


def wordCloud_generator(review2, title=None):
    wordcloud = WordCloud(width = 800, height = 800,
                          background_color ='white',
                          min_font_size = 10
                         ).generate(" ".join(review2.values))                      
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=30)
    plt.show()
wordCloud_generator(review2['comments'], title="Top words in reviews")

**Dataset 3: Calendar**

In [ ]:
calendar.info()

In [ ]:
#change the date column to a datetime 
calendar['date'] = pd.to_datetime(calendar.date)
calendar.info(verbose=True, null_counts=True)

In [ ]:
#missing values in Calender data

plt.figure(figsize=(10,6));
null_price = calendar.isnull().sum()
(null_price/calendar.shape[0]).plot(kind='bar');

In [ ]:
#removing null dates
calendar.dropna(axis=0,subset=['price'],inplace=True)

In [ ]:
#data prep/wrangling

#Calender dataset

# change the date column to a datetime 
calendar['date'] = pd.to_datetime(calendar.date)
calendar.info(verbose=True, null_counts=True)

In [ ]:
# drop the missing values in price and drop 
calendar= calendar.dropna(subset=['price'], axis = 0)
calendar.info(verbose=True, null_counts=True)

In [ ]:
#function to clean and convert str columns to numeric

def str_to_num(df,column):
    df[column] = pd.to_numeric(df[column].apply(lambda x : str(x).replace('$','').replace(",",'')),errors='coerce')
    return df

In [ ]:
columns = ['price','adjusted_price']

for col in columns:
    calendar = str_to_num(calendar,col)

calendar[columns][:2]

In [ ]:
# to clean the price and convert it into a float 
calendar = str_to_num(calendar,'price')
calendar.info(verbose=True, null_counts=True)

In [ ]:
calendar.describe()

In [ ]:
# add month and year column to the calender dataset
calendar['month'], calendar['year'] = calendar.date.dt.month, calendar.date.dt.year
calendar.info(verbose=True, null_counts=True)

In [ ]:
#effect on booking prices at different times of the year

calendar.available.value_counts()


In [ ]:
#We can observe here that there is a price hike in mid of the year and prices are lowest at the start of the year.
import plotly.offline as pyoff

price = pd.DataFrame(calendar.groupby(['month','available']).mean()['price'].reset_index())

data = [
    go.Scatter(
        x = price['month'],
        y = price.price,
        name = 'Price'
    )
]

layout = go.Layout(
    title = 'Booking prices as per months',
    xaxis = dict(title='Months'),
    yaxis = dict(title= '$ Price'),
    showlegend=True,
    
)
fig = go.Figure(data=data,layout=layout)

pyoff.iplot(fig)

In [ ]:
#as per the dataset all listings are available

available_count_daily = calendar.groupby('date').count()[['price']]
available_count_daily = available_count_daily.rename({"price":"total_available_houses"},axis='columns')

average_price_daily = calendar.groupby('date').mean()[['price']]
# change column name
average_price_daily = average_price_daily.rename({"price":"average_prices"},axis='columns')

In [ ]:
# plot total available houses and average prices in one figure
f, ax = plt.subplots(figsize=(15, 6))
plt1 = sns.lineplot(x = available_count_daily.index,y = 'total_available_houses', 
                  data = available_count_daily,color="red",legend=False,label='No. of houses available')

ax2 = ax.twinx()
plt2 = sns.lineplot(x = average_price_daily.index,y = 'average_prices',
             data=average_price_daily,color = 'black', ax=ax2,linestyle=':', legend=False,label='Daily prices')
ax.set_title('Comparing the daily availability of airbnb listing with the daily listing prices');
ax2.legend();
ax.legend();

In [ ]:
features = calendar.columns[:-1].tolist()
print(calendar.shape)

In [ ]:
sns.set(style="whitegrid", font_scale=1.2)
plt.subplots(figsize = (8,8))
sns.countplot('available',data=calendar).set_title('Available for f and t')

In [ ]:
title = 'Availabilty for t and f'
plt.figure(figsize=(10,6))
sns.scatterplot(calendar.maximum_nights,calendar.minimum_nights,hue=calendar.available).set_title(title)
plt.ioff()

In [ ]:
#Minimum Nights

(calendar.groupby('minimum_nights')['price'].mean().sort_values(ascending=False)[:20]).plot(kind="bar", figsize=(16,8));
plt.title("Average Listing price");
plt.xlabel('Average Price $');
plt.ylabel('minimum nights');
plt.xticks(rotation=60);

In [ ]:
#Maximum nights

(calendar.groupby('maximum_nights')['price'].mean().sort_values(ascending=False)[:50]).plot(kind="bar", figsize=(16,8));
plt.title("Average Listing price");
plt.xlabel('Average Price $');
plt.ylabel('Maximum nights');
plt.xticks(rotation=50);

**Dataset 4 and 5: Airbnb Listing 1 and 2**

These two datasets were chosen mostly to show visualisation and offer marketing insights and trends. 
They will be merged together. 
Sicne they had Price there are some predictions in the end, just for testing purposes and better understandingof the data. 

In [ ]:
airbnb_listing_1.info()

In [ ]:
airbnb_listing_1.head(10)

In [ ]:
airbnb_listing_2.info()

In [ ]:
airbnb_listing_2.head(10)

In [ ]:
#checking the size of the dataset, rows and columns
airbnb_listing_1.shape

In [ ]:
#checking the size of the dataset, rows and columns
airbnb_listing_2.shape

In [ ]:
#A resultant dataframe which has the rows from the target dataframe and a new row appended.

airbnb1 = pd.DataFrame(airbnb_listing_1)

airbnb1

airbnb2 = pd.DataFrame(airbnb_listing_2)

airbnb2

In [ ]:
#The append() method in python adds a single item to the existing list. 
#It doesn't return a new list of items but will modify the original list by adding the item to the end of the list. 
#After executing the method append on the list the size of the list increases by one.

#ignore index makes it not repeat the index. 


airbnb_final = airbnb1.append(airbnb2, ignore_index = True)

airbnb_final

In [ ]:
#confirming the rows have been added to the the first dataset, now called airbnb_final. 

airbnb_final.shape

In [ ]:
airbnb_final.isnull().sum()

In [ ]:
airbnb_final.columns

In [ ]:
airbnb_final[airbnb_final.duplicated()]

In [ ]:
#Althought the values of the row of one the dataset were duplicated, we have decided to leave it here.
#The datset were soureced from different websites and we left to showcase the skills of identifying this in large datasets. 

#removing the duplicated columns

#Keeping the last duplicate value, so that it removes just the one that is duplicated. 
airbnb_final.drop_duplicates(keep='last',inplace=True)

#airbnb_final = airbnb_final.loc[:,~airbnb_final.columns.duplicated()]


In [ ]:
#checking the shape of the dataset
airbnb_final.shape

In [ ]:
#dropped 36 columns

#some of the rationale include privacy concerns: photo URL, addresses and PII. 

airbnb_clean = airbnb_final.drop(columns=['Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host URL', 'Host Name', 'Calendar Updated', 'Calendar last Scraped',
       'Host About','Host Thumbnail Url', 'Host Picture Url', 'Host Verifications', 'Street',
       'Has Availability', 'Availability 30', 'Availability 60',
       'Availability 90', 'First Review', 'Last Review','License', 'Jurisdiction Names',
       'Cancellation Policy', 'Calculated host listings count'])

airbnb_clean.head()

In [ ]:
airbnb_clean.isnull().sum()

In [ ]:
airbnb_clean.info()

In [ ]:
#replacing all NaN values with 0 for categorical data
airbnb_clean.fillna({'Host Location':0}, inplace=True)
airbnb_clean.fillna({'Host Response Time':0}, inplace=True)
airbnb_clean.fillna({'Host Acceptance Rate':0}, inplace=True)
airbnb_clean.fillna({'Host Neighbourhood':0}, inplace=True)
airbnb_clean.fillna({'Neighbourhood':0}, inplace=True)
airbnb_clean.fillna({'Neighbourhood Group Cleansed':0}, inplace=True)
airbnb_clean.fillna({'City':0}, inplace=True)
airbnb_clean.fillna({'State':0}, inplace=True)
airbnb_clean.fillna({'Zipcode':0}, inplace=True)
airbnb_clean.fillna({'Market':0}, inplace=True)
airbnb_clean.fillna({'Amenities':0}, inplace=True)
airbnb_clean.fillna({'Features':0}, inplace=True)


In [ ]:
airbnb_clean.isnull().sum()

In [ ]:
#The first chosen technique is one of the possible ways of dealing with null values by using median 
#to impute the missing values.
#Median is better than mean, because mean is influenced by the outliers. 

missing_col = ['Host Response Rate']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Bathrooms']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Bedrooms']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Beds']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Square Feet']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Price']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Weekly Price']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Monthly Price']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Security Deposit']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Cleaning Fee']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Review Scores Rating']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Review Scores Accuracy']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Review Scores Cleanliness']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Review Scores Checkin']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Review Scores Communication']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()


missing_col = ['Review Scores Location']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Review Scores Value']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

missing_col = ['Reviews per Month']
for i in missing_col:
 airbnb_clean.loc[airbnb_clean.loc[:,i].isnull(),i]=airbnb_clean.loc[:,i].median()

In [ ]:
#Confirming the datset does not have any null values, they have been replaced by the median. 

airbnb_clean.isnull().sum()

**Data Visualisation**

In [ ]:
airbnb_clean.head(10)

In [ ]:
airbnb_clean.rename(columns = {'Neighbourhood Cleansed':'NeighbourhoodCleansed'}, inplace = True)
airbnb_clean.rename(columns = {'Neighbourhood Group Cleansed':'NeighbourhoodGroupCleansed'}, inplace = True)
airbnb_clean.rename(columns = {'Room Type':'RoomType'}, inplace = True)
airbnb_clean.rename(columns = {'Availability 365':'Availability365'}, inplace = True)
airbnb_clean.rename(columns = {'Minimum Nights':'MinimumNights'}, inplace = True)
airbnb_clean.rename(columns = {'Number of Reviews':'NumberofReviews'}, inplace = True)

In [ ]:
#The dataset contains locations we are not interested in:

# Drop a row by condition
airbnb_clean[airbnb_clean.NeighbourhoodCleansed != 'Dublin City']

In [ ]:
#After removing the cities not relevant for the project, there are 5377 observations to work with. 

airbnb_dub = airbnb_clean[airbnb_clean.NeighbourhoodCleansed == "Dublin City"]


In [ ]:
airbnb_dub.head()

In [ ]:
airbnb_dub.shape

In [ ]:
airbnb_dub.columns

In [ ]:
#catplot room type and price
sns.catplot(x="RoomType", y="Price", data=airbnb_dub);

In [ ]:
airbnb_dub['Neighbourhood'].tolist()


In [ ]:
#https://www.wikiwand.com/en/List_of_Dublin_postal_districts

#separating per area


def getArea(Neighbourhood):
    if (Neighbourhood == "Artane" or Neighbourhood == "Fairview" or Neighbourhood == "Stoneybatter/Arbour Hill" or Neighbourhood == "Smithfield" or Neighbourhood == "Phoenix Park and Surrounds" or Neighbourhood == "The Liberties" or Neighbourhood == "Cabra" or Neighbourhood == "Kilmainham" or Neighbourhood == "Raheny" or Neighbourhood == "East Wall" or Neighbourhood == "Inchicore" or Neighbourhood == "Glasnevin" or Neighbourhood == "Addison Park" or Neighbourhood == "Donnycarney" or Neighbourhood == "Marino" or Neighbourhood == "Pelletstown"):
       return "North Central Area"
    elif(Neighbourhood == "" or Neighbourhood == "Ballymun" or Neighbourhood == "Santry" or Neighbourhood == "Finglas"):
       return "North West Area"
    elif(Neighbourhood == "Clontarf" or Neighbourhood == "Drumcondra" or Neighbourhood == "Phibsborough"):
       return "North East Area"
    elif(Neighbourhood =="Ringsend/Irishtown" or Neighbourhood=="North City Central/O'Connell Street" or Neighbourhood == "Old City" or Neighbourhood == "Docklands" or Neighbourhood == "Rialto" or Neighbourhood == "Trinity College" or Neighbourhood == "Temple Bar" or Neighbourhood == "Georgian Quarter"):
       return "Central Area"
    elif(Neighbourhood =="Portobello" or Neighbourhood=="Ranelagh and Rathmines" or Neighbourhood == "Ballsbridge" or Neighbourhood == "Donnybrook" or Neighbourhood == "Dartry" or Neighbourhood == "Ardilea"):
       return "South Central Area"
    elif(Neighbourhood =="Rathgar" or Neighbourhood=="Sandymount" or Neighbourhood == "Booterstown"):
       return "South East Area"
    elif(Neighbourhood =="Crumlin" or Neighbourhood=="Kimmage" or Neighbourhood == "Harold's Cross" or Neighbourhood == "Ballyfermot" or Neighbourhood == "Terenure"):
       return "South West Area"
    else:
       return "undefined"


airbnb_dub['area'] = airbnb_dub['Neighbourhood'].apply(getArea)
airbnb_dub.head()



In [ ]:
#catplot neighbourhood_group and price
sns.catplot(x="area", y="Price", kind="boxen",
            data=airbnb_dub);


In [ ]:
# create countplot roomtype and neighbourhood type

#change to piechart


plt.figure(figsize=(10,10))
sns.countplot(airbnb_dub['RoomType'],hue=airbnb_dub['area'], palette='plasma')

In [ ]:
#https://matplotlib.org/3.1.1/gallery/pie_and_polar_charts/pie_features.html#sphx-glr-gallery-pie-and-polar-charts-pie-features-py

labels = airbnb_dub.area.value_counts().index
colors = ['#008fd5','#fc4f30','#e5ae38','#6d904f','#8b8b8b']
explode = (0.1,0,0,0,0,0,0,0)
shape = airbnb_dub.area.value_counts().values
plt.figure(figsize=(15,15))
plt.pie(shape, explode = explode, labels=shape, colors= colors, autopct = '%1.1f%%', startangle=90)
plt.legend(labels)
plt.title('Neighbourhood Area')
plt.show()


In [ ]:
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot


contt = ff.create_table(pd.crosstab(airbnb_dub.Neighbourhood, airbnb_dub.RoomType), index=True)
iplot(contt)

In [ ]:
plt.figure(figsize=(22,10))

plt.subplot(1,2,1)
plt.title('Price Distribution Plot')
sns.distplot(airbnb_dub.Price)

plt.subplot(1,2,2)
plt.title('Price Spread')
sns.boxplot(y=airbnb_dub.Price)

plt.show()

#Distribution: graph of price distribution is positive skewed, and outliers.


In [ ]:
from scipy import stats
z = np.abs(stats.zscore(airbnb_dub['Price']))
print(z)


In [ ]:
#Skewness of the price i have used the absolute values of z score
#and after set the threshhold as 3 it means that above the value of 3 all the 
#values will considered as outlier and then i have used the skew function to check the skewness of price data.


threshold = 3
print(np.where(z > 3))

In [ ]:
from scipy.stats import skew
print(skew(airbnb_dub.Price))

In [ ]:
sns.countplot(airbnb_dub.RoomType)


In [ ]:
import plotly.express as px

fig = px.violin(airbnb_dub, y="Price", x="area", points="all")
fig.show()

In [ ]:
plt.figure(figsize=(15,15))
sns.set(style="whitegrid")

ax = sns.barplot(x="area", y="Price",data=airbnb_dub)

In [ ]:
plt.figure(figsize=(9,8))
sns.set(style="whitegrid")

ax = sns.countplot(x="RoomType", hue="area",data=airbnb_dub)

In [ ]:
north_central = airbnb_dub[airbnb_dub.area=="North Central Area"].Availability365
north_west = airbnb_dub[airbnb_dub.area=="North West Area"].Availability365
north_east = airbnb_dub[airbnb_dub.area=="North East Area"].Availability365
central = airbnb_dub[airbnb_dub.area=="Central Area"].Availability365
south_central = airbnb_dub[airbnb_dub.area=="South Central Area"].Availability365
south_east = airbnb_dub[airbnb_dub.area=="South East Area"].Availability365
south_west = airbnb_dub[airbnb_dub.area=="South West Area"].Availability365


fig = go.Figure()
# Use x instead of y argument for horizontal plot
fig.add_trace(go.Box(x=north_central, name='North Central'))
fig.add_trace(go.Box(x=north_west, name='North West'))
fig.add_trace(go.Box(x=north_east, name='North East'))
fig.add_trace(go.Box(x=central, name='Central'))
fig.add_trace(go.Box(x=south_central, name='South Central'))
fig.add_trace(go.Box(x=south_east, name='South East'))
fig.add_trace(go.Box(x=south_west, name='South West'))

fig.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(airbnb_dub.Longitude,airbnb_dub.Latitude,hue=airbnb_dub.area)
plt.ioff()

In [ ]:
sns.clustermap(airbnb_dub.corr(),annot = False);

In [ ]:
avg_price = airbnb_dub.groupby(['area'])['Price'].mean().plot(kind='bar')


In [ ]:
airbnb_dub.columns


In [ ]:
airbnb_clean.rename(columns = {'Minimum Nights':'MinimumNights'}, inplace = True)
airbnb_clean.columns

In [ ]:

airbnb_dub.groupby(['area'])['Amenities'].count().plot(kind='bar')


In [ ]:
# Relationship between price and room type
plt.figure()
sns.scatterplot(x='RoomType', y='Price', hue="RoomType", data=airbnb_dub)
plt.xlabel("Room Type")
plt.ylabel("Price")
plt.title("Room Type vs Price", size=15, weight='bold')

In [ ]:
# Relationship between price and neighbourhood_group_cleansed
plt.figure()
sns.scatterplot(x="area", y="Price", hue="area", data=airbnb_dub)
plt.xlabel("Neighbourhood Group")
plt.ylabel("Price")
plt.title("Neighbourhood Group vs Price", size=15, weight='bold')

In [ ]:
# Relationship between price, room_type and neighbourhood_group_cleansed
plt.figure()
sns.scatterplot(x="RoomType", y="Price", hue="area", size="area", sizes=(50, 200),
               data=airbnb_dub)
plt.xlabel("Room Type")
plt.ylabel("Price")
plt.title("Room Type vs Price vs Neighbourhood Group",size=15, weight='bold')

In [ ]:
airbnb_clean.rename(columns = {'Number of Reviews':'NumberofReviews'}, inplace = True)
airbnb_clean.NumberofReviews

In [ ]:
# Plot first 1000 listings with most number of reviews on the map using folium

mostReviewsDubData = airbnb_dub.sort_values(by=['NumberofReviews'], ascending=False).head(1000)

In [ ]:
import folium
from folium import plugins
from folium.plugins import MarkerCluster

print('Rooms with the most number of reviews')
Lat = 53.3498
Long = -6.266155

mapdf1=folium.Map([Lat,Long],zoom_start=10,)

mapdf1_rooms_map=plugins.MarkerCluster().add_to(mapdf1)

for lat,lon,label in zip(mostReviewsDubData.Latitude,mostReviewsDubData.Longitude,mostReviewsDubData.area):
    folium.Marker(location=[lat,lon],icon=folium.Icon(icon='home'),popup=label).add_to(mapdf1_rooms_map)

mapdf1.add_child(mapdf1_rooms_map)

mapdf1

In [ ]:
#Find 1000 most expensive rooms
mostExpensiveDub=airbnb_dub.sort_values(by=['Price'],ascending=False).head(1000)

In [ ]:
print('Most Expensive rooms')
Lat = 53.3498
Long = -6.266155

mapdf1=folium.Map([Lat,Long],zoom_start=10,)

mapdf1_rooms_map=plugins.MarkerCluster().add_to(mapdf1)

for lat,lon,label in zip(mostExpensiveDub.Latitude,mostExpensiveDub.Longitude,mostExpensiveDub.area):
    folium.Marker(location=[lat,lon],icon=folium.Icon(icon='home'),popup=label).add_to(mapdf1_rooms_map)
mapdf1.add_child(mapdf1_rooms_map)

mapdf1

In [ ]:
import folium
from folium.plugins import HeatMap
map_folium = folium.Map([Lat, Long],zoom_start=11.4)
HeatMap(airbnb_dub[['Latitude','Longitude']].dropna(),radius=8,gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(map_folium)
display(map_folium)



In [ ]:
#pip install folium

In [ ]:
import folium

m=folium.Map(
    location=[airbnb_dub["Latitude"].mean(), airbnb_dub["Longitude"].mean()],
    zoom_start=6)

airbnb_dub.apply(
    lambda row: folium.Marker(
        location=[row['Latitude'], row['Longitude']]
        ).add_to(m),
    axis=1)
m

### Price Prediction

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import r2_score
import numpy as np

In [ ]:
#need to add more features maybe ? 
features = airbnb_dub[['Accommodates', 
                     'Bathrooms', 'Beds', 'Beds', 'Price', 'NumberofReviews']]

In [ ]:
features.info()

In [ ]:
fitters = features.query('Price <= 1000')
fitters.shape

In [ ]:

y = fitters['Price']
X = fitters.drop(columns = {'Price'}, axis = 1)
X.shape

In [ ]:
# Train Test Split
# 80-20 Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
import time
# Train Linear Regression model
lm = LinearRegression()
t0=time.time()
lm.fit(X_train,y_train)
print("Training time:", round(time.time()-t0, 3), "s") # the time would be round to 3 decimal in seconds
t1=time.time()
pred_linear_x = lm.predict(X_test)
print("Prediction time:", round(time.time()-t1, 3), "s")

error_airbnb = pd.DataFrame({
        'Actual Values': np.array(y_test).flatten(),
        'Predicted Values': pred_linear_x.flatten()})
print(error_airbnb.head().to_string())

In [ ]:
print('MAE: %f'% mean_absolute_error(y_test, pred_linear_x))
print('RMSE: %f'% np.sqrt(mean_squared_error(y_test, pred_linear_x)))   
print('R2 %f' % r2_score(y_test, pred_linear_x))

In [ ]:
# Train Ridge Regression model
ridge_x = Ridge(alpha = 0.01, normalize = True)
t0=time.time()
ridge_x.fit(X_train,y_train) 
print("Training time:", round(time.time()-t0, 3), "s") # the time would be round to 3 decimal in seconds
t1=time.time()
pred_ridge_x = ridge_x.predict(X_test) 
print("Prediction time:", round(time.time()-t1, 3), "s")

error_airbnb = pd.DataFrame({
        'Actual Values': np.array(y_test).flatten(),
        'Predicted Values': pred_ridge_x.flatten()})
print(error_airbnb.head().to_string())

In [ ]:
print('MAE: %f'% mean_absolute_error(y_test, pred_ridge_x))
print('RMSE: %f'% np.sqrt(mean_squared_error(y_test, pred_ridge_x)))   
print('R2 %f' % r2_score(y_test, pred_ridge_x))

In [ ]:
# Train Lasso Regression model
Lasso_x = Lasso(alpha = 0.001, normalize =False)
t0=time.time()
Lasso_x.fit(X_train,y_train)
print("Training time:", round(time.time()-t0, 3), "s") # the time would be round to 3 decimal in seconds
t1=time.time()
pred_Lasso_x = Lasso_x.predict(X_test) 
print("Prediction time:", round(time.time()-t1, 3), "s")

error_airbnb = pd.DataFrame({
        'Actual Values': np.array(y_test).flatten(),
        'Predicted Values': pred_Lasso_x.flatten()})
print(error_airbnb.head().to_string())

In [ ]:
print('MAE: %f' % mean_absolute_error(y_test, pred_Lasso_x))
print('RMSE: %f' % np.sqrt(mean_squared_error(y_test, pred_Lasso_x)))
print('R2 %f' % r2_score(y_test, pred_Lasso_x))

**Dataset 6: Rate**

In [ ]:
rate.head()

In [ ]:
#Based on the information on the dataset itself, I added these columns names. 
rate.columns = [
    "Listing URL",
    "Property Type",
    "Latitude",
    "Longitude",
    "Star Rating",
    "Number of Active Days",
    "Bedrooms",
    "Has pool",
    "Cleaning Fee",
    "Extra Guest Fee",
    "Daily Rate (2020-11)",
    "Daily Rate (2020-12)",
    "Daily Rate (2021-01)",
    "Daily Rate (2021-02)",
    "Daily Rate (2021-03)",
    "Daily Rate (2021-04)",
    "Daily Rate (2021-05)",
    "Daily Rate (2021-06)",
    "Daily Rate (2021-07)",
    "Daily Rate (2021-08)",
    "Daily Rate (2021-09)",
    "Daily Rate (2021-10)",
    "Occupancy Rate (2020-11)",
    "Occupancy Rate (2020-12)",
    "Occupancy Rate (2021-01)",
    "Occupancy Rate (2021-02)",
    "Occupancy Rate (2021-03)",
    "Occupancy Rate (2021-04)",
    "Occupancy Rate (2021-05)",
    "Occupancy Rate (2021-06)",
    "Occupancy Rate (2021-07)",
    "Occupancy Rate (2021-08)",
    "Occupancy Rate (2021-09)",
    "Occupancy Rate (2021-10)",
    "Revenue (2020-11)",
    "Revenue (2020-12)",
    "Revenue (2021-01)",
    "Revenue (2021-02)",
    "Revenue (2021-03)",
    "Revenue (2021-04)",
    "Revenue (2021-05)",
    "Revenue (2021-06)",
    "Revenue (2021-07)",
    "Revenue (2021-08)",
    "Revenue (2021-09)",
    "Revenue (2021-10)",
]


In [ ]:
rate.head()

In [ ]:
#dropping the two first rows.
rate.drop(index=rate.index[0:2], 
        axis=0, 
        inplace=True)

In [ ]:
#replacing all NaN values with 0
rate.fillna({'Cleaning Fee':0}, inplace=True)
rate.fillna({'Extra Guest Fee':0}, inplace=True)

In [ ]:
rate.isnull().sum()

In [ ]:
#examine the dataset
(rate[['Star Rating', 'Number of Active Days', 'Daily Rate (2020-12)', 'Occupancy Rate (2020-12)',
       'Revenue (2020-12)']]
 .describe())

In [ ]:
#need to be encoded to integer

rate.info()


In [ ]:
#separating the dataset into categorical and numeric

#later to change the object type to float64

categorical_col = ['Listing URL','Property Type', 'Has pool']

numeric_col = ['Latitude', 
               'Longitude', 
               'Star Rating', 
               'Number of Active Days', 
               'Bedrooms', 
               'Cleaning Fee', 
               'Extra Guest Fee',            
               'Daily Rate (2020-11)',         
               'Daily Rate (2020-12)',
               'Daily Rate (2021-01)',
               'Daily Rate (2021-02)',        
               'Daily Rate (2021-03)',       
                'Daily Rate (2021-04)',        
                'Daily Rate (2021-05)',        
                'Daily Rate (2021-06)',        
                'Daily Rate (2021-07)',        
                'Daily Rate (2021-08)',        
                'Daily Rate (2021-09)',        
                'Daily Rate (2021-10)',        
                'Occupancy Rate (2020-11)',    
                'Occupancy Rate (2020-12)',   
                'Occupancy Rate (2021-01)',    
                'Occupancy Rate (2021-02)',    
                'Occupancy Rate (2021-03)',    
                'Occupancy Rate (2021-04)',    
                'Occupancy Rate (2021-05)',    
                'Occupancy Rate (2021-06)',    
                'Occupancy Rate (2021-07)',    
                'Occupancy Rate (2021-08)',    
                'Occupancy Rate (2021-09)',    
                'Occupancy Rate (2021-10)',   
                'Revenue (2020-11)',           
                'Revenue (2020-12)',         
                'Revenue (2021-01)',           
                'Revenue (2021-02)',           
                'Revenue (2021-03)',           
                'Revenue (2021-04)',           
                'Revenue (2021-05)',           
                'Revenue (2021-06)',           
                'Revenue (2021-07)',           
                'Revenue (2021-08)',           
                'Revenue (2021-09)',           
                'Revenue (2021-10)',
              ]


In [ ]:
print(numeric_col)

In [ ]:
rate[numeric_col] = rate[numeric_col].apply(pd.to_numeric, errors='coerce')

In [ ]:
rate.info()

In [ ]:
#NOT NEEDED - things I tried.
#encoding the dataset so that I can add columns for the rates values. 
#rate['Cleaning Fee'] = rate['Cleaning Fee'].astype(float)

In [ ]:
#rate.drop(columns=['Listing URL', 'Property Type', 'Latitude', 'Longitude', 'Has pool'])

In [ ]:
rate.head()

In [ ]:
#rate = rate.drop(['Daily Rate (2021-01)','Daily Rate (2021-02)', 'Daily Rate (2021-03)','Daily Rate (2021-04)', 'Daily Rate (2021-05)','Daily Rate (2021-06)', 'Daily Rate (2021-07)','Daily Rate (2021-08)', 'Daily Rate (2021-09)','Daily Rate (2021-10)', 'Daily Rate (2020-11)','Daily Rate (2020-12)','Revenue (2020-11)','Revenue (2020-12)','Revenue (2021-01)','Revenue (2021-02)','Revenue (2021-03)','Revenue (2021-04)','Revenue (2021-05)','Revenue (2021-06)','Revenue (2021-07)','Revenue (2021-08)','Revenue (2021-09)','Revenue (2021-10)','Occupancy Rate (2020-11)','Occupancy Rate (2020-12)','Occupancy Rate (2021-01)','Occupancy Rate (2021-02)','Occupancy Rate (2021-03)','Occupancy Rate (2021-04)','Occupancy Rate (2021-05)','Occupancy Rate (2021-06)','Occupancy Rate (2021-07)','Occupancy Rate (2021-08)','Occupancy Rate (2021-09)','Occupancy Rate (2021-10)'], axis=1)

In [ ]:
#del rate['Daily Rate (2021-01)','Daily Rate (2021-02)', 'Daily Rate (2021-03)','Daily Rate (2021-04)', 'Daily Rate (2021-05)','Daily Rate (2021-06)', 'Daily Rate (2021-07)','Daily Rate (2021-08)', 'Daily Rate (2021-09)','Daily Rate (2021-10)', 'Daily Rate (2020-11)','Daily Rate (2020-12)','Revenue (2020-11)','Revenue (2020-12)','Revenue (2021-01)','Revenue (2021-02)','Revenue (2021-03)','Revenue (2021-04)',          'Revenue (2021-05)','Revenue (2021-06)','Revenue (2021-07)','Revenue (2021-08)','Revenue (2021-09)','Revenue (2021-10)','Occupancy Rate (2020-11)','Occupancy Rate (2020-12)','Occupancy Rate (2021-01)','Occupancy Rate (2021-02)','Occupancy Rate (2021-03)','Occupancy Rate (2021-04)','Occupancy Rate (2021-05)','Occupancy Rate (2021-06)','Occupancy Rate (2021-07)','Occupancy Rate (2021-08)','Occupancy Rate (2021-09)','Occupancy Rate (2021-10)']

In [ ]:
#Check how many properties have a pool
rate['Has pool'].value_counts()

In [ ]:
rate['Cleaning Fee'].value_counts()

In [ ]:
rate['Extra Guest Fee'].describe()

In [ ]:
#2020-11
rate['Occupancy Rate (2020-11)'].value_counts()

In [ ]:
#2020-11
rate['Daily Rate (2020-11)'].value_counts()

In [ ]:
#2020-11
rate['Revenue (2020-11)'].value_counts()

In [ ]:
#2021-10
rate['Occupancy Rate (2021-10)'].value_counts()

In [ ]:
#2021-10
rate['Daily Rate (2021-10)'].value_counts(ascending=False)

In [ ]:
#2021-10
rate['Revenue (2021-10)'].value_counts(ascending=False)

In [ ]:
#Pair Plot
#sns.pairplot(rate)
#plt.figure(figsize=(10,7))

In [ ]:
plt.figure(figsize=(10,6))
rate.Bedrooms.hist();
plt.title('Bedroom distribution');
plt.xlabel('Number of Bedrooms');
plt.ylabel('Count');

In [ ]:
rate.info()

In [ ]:
#creating this second dataset so that we can try the predictions without compromsing the original dataset. 

rate2 = rate.copy()

In [ ]:
#mean all the Revenue, Occupancy and Daily Rate

rate['daily_rate_mean'] = rate[['Daily Rate (2021-01)','Daily Rate (2021-02)', 'Daily Rate (2021-03)','Daily Rate (2021-04)', 'Daily Rate (2021-05)','Daily Rate (2021-06)', 'Daily Rate (2021-07)','Daily Rate (2021-08)', 'Daily Rate (2021-09)','Daily Rate (2021-10)', 'Daily Rate (2020-11)','Daily Rate (2020-12)']].mean(axis=1)

rate['revenue_mean'] = rate[['Revenue (2020-11)','Revenue (2020-12)','Revenue (2021-01)','Revenue (2021-02)','Revenue (2021-03)','Revenue (2021-04)',          'Revenue (2021-05)','Revenue (2021-06)','Revenue (2021-07)','Revenue (2021-08)','Revenue (2021-09)','Revenue (2021-10)']].mean(axis=1)

rate['occupancy_mean'] = rate[['Occupancy Rate (2020-11)','Occupancy Rate (2020-12)','Occupancy Rate (2021-01)','Occupancy Rate (2021-02)','Occupancy Rate (2021-03)','Occupancy Rate (2021-04)','Occupancy Rate (2021-05)','Occupancy Rate (2021-06)','Occupancy Rate (2021-07)','Occupancy Rate (2021-08)','Occupancy Rate (2021-09)','Occupancy Rate (2021-10)']].mean(axis=1)


In [ ]:
rate = rate.drop(['Daily Rate (2021-01)','Daily Rate (2021-02)', 'Daily Rate (2021-03)','Daily Rate (2021-04)', 'Daily Rate (2021-05)','Daily Rate (2021-06)', 'Daily Rate (2021-07)','Daily Rate (2021-08)', 'Daily Rate (2021-09)','Daily Rate (2021-10)', 'Daily Rate (2020-11)','Daily Rate (2020-12)'], axis=1)
rate = rate.drop(['Revenue (2020-11)','Revenue (2020-12)','Revenue (2021-01)','Revenue (2021-02)','Revenue (2021-03)','Revenue (2021-04)','Revenue (2021-05)','Revenue (2021-06)','Revenue (2021-07)','Revenue (2021-08)','Revenue (2021-09)','Revenue (2021-10)'], axis=1)
rate = rate.drop(['Occupancy Rate (2020-11)','Occupancy Rate (2020-12)','Occupancy Rate (2021-01)','Occupancy Rate (2021-02)','Occupancy Rate (2021-03)','Occupancy Rate (2021-04)','Occupancy Rate (2021-05)','Occupancy Rate (2021-06)','Occupancy Rate (2021-07)','Occupancy Rate (2021-08)','Occupancy Rate (2021-09)','Occupancy Rate (2021-10)'], axis=1)

                  

In [ ]:
rate.columns

In [ ]:
#correlations - example 

rate.corr() # Whole correlation matrix
rate.corr()['revenue_mean'] # Check correlations with outcome only
sns.heatmap(rate.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
#correlations - example 

rate.corr() # Whole correlation matrix
rate.corr()['daily_rate_mean'] # Check correlations with outcome only
sns.heatmap(rate.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
#correlations - example 

rate.corr() # Whole correlation matrix
rate.corr()['occupancy_mean'] # Check correlations with outcome only
sns.heatmap(rate.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
# Import packages
import pandas as pd
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import lars_path
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score
import scipy.stats as stats
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sns.distplot(rate['daily_rate_mean'], kde=True,);
fig = plt.figure()
res = stats.probplot(rate['daily_rate_mean'], plot=plt)
print("Skewness: %f" % rate['daily_rate_mean'].skew())
print("Kurtosis: %f" % rate['daily_rate_mean'].kurt())

#Need to review this as the dataset looks skewed 

**Predicting the Occupancy Rate**

In [ ]:
rate

In [ ]:
rate.isnull().sum()

In [ ]:
#If we have time we can merge rate and listing together for a prediction based on
#https://www.kaggle.com/code/brittabettendorf/what-factors-influence-demand-eda-occupancy
rate.fillna({'Bedrooms': '0'}, inplace = True)

In [ ]:
#Dropping not needed columns for predicting the rate
rate.drop(['Listing URL', 'Property Type', 'Has pool'], axis=1, inplace = True)

In [ ]:
X = rate.drop(['occupancy_mean'], axis=1).values
y = rate['occupancy_mean'].values

In [ ]:
print(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=0)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)
print(y_pred)

In [ ]:
#I am trying different possibilities using different variables, at the moment this one is only the occupancy_mean
r2_score(y_test, y_pred)

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(y_test, y_pred)
plt.xlabel('Occupancy at the moment')
plt.ylabel('Occupancy predicted')
plt.title('Occupancy rate')

In [ ]:
#Predicted values
occu_rate = pd.DataFrame({'Current occupancy': y_test, 'Predicted occupancy': y_pred, 'Profit/Loss': y_test - y_pred})
occu_rate

In [ ]:
#using the second dataset rate2 for another trial

In [ ]:
rate2

In [ ]:
rate2.drop(['Extra Guest Fee', 'ExtravGuest Fee','Cleaning Fee', 'Daily Rate (2021-01)','Daily Rate (2021-02)', 'Daily Rate (2021-03)','Daily Rate (2021-04)', 'Daily Rate (2021-05)','Daily Rate (2021-06)', 'Daily Rate (2021-07)','Daily Rate (2021-08)', 'Daily Rate (2021-09)','Daily Rate (2021-10)', 'Daily Rate (2020-11)','Daily Rate (2020-12)','Revenue (2020-11)','Revenue (2020-12)','Revenue (2021-01)','Revenue (2021-02)','Revenue (2021-03)','Revenue (2021-04)','Revenue (2021-05)','Revenue (2021-06)','Revenue (2021-07)','Revenue (2021-08)','Revenue (2021-09)','Revenue (2021-10)' ''], axis=1, inplace = True)

In [ ]:
rate2.drop(['Listing URL', 'Property Type'], axis =1, inplace = True)

In [ ]:
rate2.drop(['Has pool'], axis = 1, inplace = True)

In [ ]:
rate2.isnull().sum()

In [ ]:
rate2.fillna({'Bedrooms': '0'}, inplace = True)

In [ ]:
X = rate2.drop(['Occupancy Rate (2020-11)'], axis=1).values
y = rate2['Occupancy Rate (2020-11)'].values

In [ ]:
print(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state=0)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
#I am trying different possibilities using different variables, at the moment this one is only the occupancy_mean
r2_score(y_test, y_pred)

In [ ]:
lr.score(X_train, y_train)

In [ ]:
plt.figure(figsize=(8,4))
plt.scatter(y_test, y_pred)
plt.xlabel('Occupancy at the moment')
plt.ylabel('Occupancy predicted')
plt.title('Occupancy rate')

In [ ]:
#Predicted values
occu_rate = pd.DataFrame({'Current occupancy': y_test, 'Predicted occupancy': y_pred, 'Profit/Loss': y_test - y_pred})
occu_rate

In [ ]:
from sklearn import linear_model

lasso_reg = linear_model.Lasso(alpha=50, max_iter=100, tol=0.1)

lasso_reg.fit(X_train, y_train)

In [ ]:
lasso_reg.score(X_test, y_test)

In [ ]:
lasso_reg.score(X_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=50, max_iter=100, tol=0.1)

ridge_reg.fit(X_train, y_train)

In [ ]:
ridge_reg.score(X_test, y_test)

In [ ]:
ridge_reg.score(X_train, y_train)

In [ ]:
#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html
#http://insideairbnb.com/ireland/
#https://public.opendatasoft.com/explore/dataset/airbnb-listings/export/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&q=ireland
#https://app.airbtics.com/airbnb-data/ireland/0/dublin
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
#https://www.codegrepper.com/code-examples/python/convert+object+to+float64+pandas
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
#https://stackoverflow.com/questions/48190843/issue-defining-kneighborsclassifier-in-jupyter-notebooks
#https://realpython.com/linear-regression-in-python/
#https://huggingface.co/docs/transformers/v4.18.0/en/installation
#https://huggingface.co/docs/transformers/tokenizer_summary
#https://realpython.com/pandas-merge-join-and-concat/
#https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
#https://stackoverflow.com/questions/49188960/how-to-show-all-columns-names-on-a-large-pandas-dataframe
#https://www.stackvidhya.com/add-row-to-dataframe/#:~:text=You%20can%20add%20rows%20to,append()
#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html#:~:text=2014-,The%20pd.,information%20from%20the%20two%20inputs.
